# *k*-Nearest Neighbours - fruit

Hogeschool Utrecht (c) 2020

Tijmen Muller (tijmen.muller@hu.nl)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix

## Data inlezen

In [ ]:
# inlezen tabel met fruit gegevens
fruits = pd.read_table('knn_fruit.txt')
print('Aantal meetwaarden {0:d}'.format(len(fruits)))
fruits.head()

De kolommen zijn als volgt:
* `fruit_label` is een identificatienummer, overeenkomend met `fruit_name`
* `fruit_name` is het fruittype
* `fruit_subtype` is het subtype (bijvoorbeeld het soort appel)
* `mass` is het gewicht in grammen
* `width` is de breedte in cm
* `height` is de hoogte in cm
* `color_score` is een waarde uit het kleuren spectrum:
  * groen: 0.45-0.65
  * geel: 0.65-0.75
  * oranje: 0.75-0.85
  * rood: 0.85-1.00

In [ ]:
# maak een dictionary van fruit_label naar fruit_name
lookup_fruit_name = dict(zip(fruits['fruit_label'].unique(),fruits['fruit_name'].unique()))
lookup_fruit_name

## Aanpak volgens werkwijze `scikit-learn`

### 1. Kies het modeltype met 2. de hyperparameters

We kiezen _k_-Nearest Neighbours (`KNeighborsClassifier`) met _k_ = 5.

In [ ]:
# Maak kNN object
knn = KNeighborsClassifier(n_neighbors = 5)

### 3. Organiseer de data

Onze feature matrix `X` bestaat uit de kolommen `mass`, `width` en `height`: dit zijn de kenmerken waar _vanuit_ we willen voorspellen.

Onze target vector `y` bestaat uit de kolom `fruit_label`: dit is het resultaat waar we _naartoe_ willen voorspellen (bij _k_-NN moet deze numeriek zijn, ookal is het een klasse).

In [ ]:
X = fruits[['mass','width','height']]
y = fruits['fruit_label']

### 4. Creëer een train- en validatieset

De methode `sklearn.model_selection.train_test_split()` deelt de feature matrix en de result vector gerandomiseerd op in een train- en een validatieset (ook wel: testset).

In [ ]:
# splits in train en test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

print(f'{len(X_train)} trainwaarden, {len(X_test)} testwaarden: {len(X_train)*100/len(fruits):.1f}%/{len(X_test)*100/len(fruits):.1f}%')


### 5. Train het model

In [ ]:
knn.fit(X_train,y_train)


### 6. Valideer het model

Het model is te valideren met de validatieset. Het model heeft de validatieset nog niet gezien (alleen de trainset is gebruikt om het model te trainen). Het model kan de resultaten voorspellen op de _feature matrix_ van de validatieset. Door de uitkomsten (voorspeld resultaat) te vergelijken met de _target vector_ van de validatieset (de échte waarden) kunnen we zien hoe goed het model voorspeld.

We kunnen de methode `sklearn.metrics.accuracy_score()` gebruiken om de voorspelde waarde en de echte waarde te vergelijken, dit geeft een percentage 'goed voorspeld'. _k_-NN heeft ook zijn eigen methode `score()`, deze doet hetzelfde.

In [ ]:
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
knn.score(X_test,y_test)

53% is natuurlijk niet zo'n goede score. Om te bepalen _hoe_ goed (of slecht) de score is, is het goed om na te denken over een baseline. Als we heel naïef/dom zouden voorspellen, wat voor score zouden we dan behalen? In dit voorbeeld zouden we bijvoorbeeld kunnen kijken welk fruittype het vaakst voorkomt en elk stuk fruit in die klasse plaatsen (best dom, toch?).

In [ ]:
fruits.groupby('fruit_name').count()

'apple' en 'orange' komen het vaakst voor, laten we 'orange' kiezen. Deze heeft `fruit_label` = 3, dus in onze 'naieve' voorspelling is het resultaat _altijd_ een 3.

In [ ]:
y_naive = np.full(len(y_pred), 3)
print(f"Resultaatvector bij 'dom' voorspellen: {y_naive}\nBaseline score: {accuracy_score(y_test, y_naive)}")

Dus als we naief voorspellen, behalen we dezelfde score! Dat is niet best...

Een andere handige referentie is de score op de *trainset*. Dit zijn de waardes _waarop_ het model heeft getraind, dus hierop zou het model natuurlijk goed moeten scoren (het model heeft zich immers op die waarden gebaseerd).

In [ ]:
y_train_pred = knn.predict(X_train)
print(f"Score bij voorspellen op de trainset: {accuracy_score(y_train, y_train_pred)}")

Dat is een veel betere voorspellingsscore -- blijkbaar voorspelt het model op de trainingswaarden wél goed en op de validatiewaarden niet. In zo'n geval zou er sprake kunnen zijn van _overfitting_.

Nu willen we natuurlijk graag inzicht  _waarom_ het model niet goed scoort. Bij classificatie kunnen we daarvoor een confusion matrix gebruiken: deze vergelijkt de voorspelde waarde met de echte waarde. We kunnen dit doen met de methode `sklearn.metrics.confusion_matrix()` (geeft een NumPy `array` terug) en vervolgens visualiseren met bijvoorbeeld Seaborn.

In [ ]:
cm = confusion_matrix(y_test, y_pred)

# Maak van de array een pandas dataframe om te visualiseren
df_cm = pd.DataFrame(cm, 
                     index = [lookup_fruit_name[i+1] for i in range(4)], 
                     columns = [lookup_fruit_name[i+1] for i in range(4)])

print(cm)
df_cm

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5), dpi=100)

ax = sns.heatmap(df_cm, annot=True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

ax.set_xlabel('voorspelde waarde')
ax.set_ylabel('echte waarde')

plt.show()

### 7. Voorspel nieuwe data

We kunnen nu het fruittype voorspellen van een nieuw meetpunt op basis van de kenmerken `mass`, `width` en `length` (met helaas maar een lage betrouwbaarheid, vanwege de lage voorspellingsscore). Bijvoorbeeld: wat voor fruittype is een nieuw stuk fruit met massa 150, breedte 6.5 en hoogte 7?

In [ ]:
fruit_feat = [150,6.5,7.0]
fruit_pred = knn.predict([fruit_feat])
print(fruit_pred)
print(f"Voorspelling: fruit_label = {fruit_pred[0]}, fruit_name = {lookup_fruit_name[fruit_pred[0]]}")

## Oefening

Bedenk verschillende aanpassingen die een beter resultaat zouden kunnen opleveren. Voer een variant uit en verifieer het resultaat.

In het voorbeeld wordt _k_ = 5 gekozen. Controleer of er verbetering is als je een andere waarde voor _k_ kiest.

In het voorbeeld worden alleen `mass`, `width`, `height` als feature gebruikt. Controleer of er verbetering is als je `color_score` toevoegt.

In het voorbeeld zijn de features niet genormaliseerd. Controleer of er verbetering is als je de features normaliseert.

In het voorbeeld wordt KNeighborsClassifier model gebruikt. Kun je met een ander model, bijv. Gaussian Naive Bayes (`GaussianNB()`) verbeteren?